In [ ]:
# pip install pandas requests beautifulsoup4

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urlparse
from datetime import date
import unicodedata

In [2]:
def clean_text(text):
    # Normalize Unicode characters
    cleaned_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return cleaned_text

In [11]:
def get_post_body(url, headers):
  response = requests.get(url, headers=headers)

  if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    section = soup.find('section', class_='main-content')

    if section:
      post_article = section.find('article', class_='post-content')
      if post_article:
        # post_container = post_article.find('div', class_='clearfix').text
        post_container = post_article.find('div', class_='clearfix')
        
        if post_container:
          post_container_text =  clean_text(post_container.get_text(separator=" ", strip=True))

          if len(post_container_text) > 10000:
            post_container_text = len(post_container_text)
            
          return post_container_text

In [12]:
def extract_date_from_url(url):
  parsed_url =  urlparse(url)
  
  path_parts = parsed_url.path.split('/')
  
  year = path_parts[1]
  month = path_parts[2]
  day = path_parts[3]
  
  date = f'{year}-{month}-{day}'
  return date

In [13]:
def get_job_postings(url, headers):
  response = requests.get(url, headers=headers)

  if response.status_code == 200:
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.content, 'html.parser')
    section = soup.find('section', class_='main-content')

    if section:
      post_container = section.find('div', class_='col-lg-8')

      job_posts = []
      
      for post in post_container.find_all('div', class_='post-classic'):
        title_heading_element = post.find('h5')

        title = clean_text(title_heading_element.find('a').text.strip())
        link = title_heading_element.find('a').get("href")

        content = get_post_body(link, headers)
        date = extract_date_from_url(link)

        job_posts.append({
          'title': title,
          'link': link,
          'content': content,
          'date': date
        })
      return job_posts
  else:
    return []

# all_jobs = get_job_postings(pagination_date_pattern, headers)
# df = pd.DataFrame(all_jobs)
# df.to_csv('job_postings.csv', index=False)

In [15]:
def aggregate_job_postings():
  headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
  }

  first_year = 2018
  first_month = 1
  first_day = 1

  year_today = 2018
  month_today = 12 
  day_today = 31
  # date_today = date.today()
  # year_today = date_today.year
  # month_today = date_today.month
  # day_today = date_today.day

  base_url = 'https://opportunitiesforyoungkenyans.co.ke'

  jobs_from_all_dates = []
  merged_jobs = []

  for year in range(first_year, year_today + 1):
    if year == year_today:
      last_month = month_today
    else:
      last_month = 12
    
    for month in range(first_month, last_month + 1):
      if year == year_today and month == month_today:
        last_day = day_today
      else:
        last_day = 31 # TODO: Adjust to accommodate different number of days per month + leap years

      for day in range(first_day, last_day + 1):
        current_date_url = f'{base_url}/{year}/{month}/{day}'
        current_date_url_response = requests.get(current_date_url, headers=headers)

        if current_date_url_response.status_code == 200:
          soup = BeautifulSoup(current_date_url_response.content, 'html.parser')
          date_url_body_tag = soup.body

          if date_url_body_tag and 'error404' in date_url_body_tag.get('class', []):
            print(f'No posts found at {current_date_url}. Skipping.')
            continue
          else:
            for page_count in range(1, 1000):
              page_url = f'{current_date_url}/page/{page_count}'
              page_url_response = requests.get(page_url, headers=headers)
              
              if page_url_response.status_code == 200:
                page_url_soup = BeautifulSoup(page_url_response.content, 'html.parser')
                page_url_body_tag = page_url_soup.body

                if page_url_body_tag and 'error404' in page_url_body_tag.get('class', []):
                  print(f'Error 404 found on {page_url}. Stopping search.')
                  break # Stop searching further pages because if current page is not found then there are no more pages for this date.
                else:
                  jobs_per_date = get_job_postings(page_url, headers)
                  jobs_from_all_dates.append(jobs_per_date)
                  print(f'Found {page_url}')
              else:
                print(f'Failed to retrieve {page_url}. Status code: {page_url_response.status_code}')
                break  # Stop searching further pages on HTTP error
            
            print(f'Day {day} over.')
          # Continue to next day
        else:
          print(f'Failed to retrieve {current_date_url}. Status code: {current_date_url_response.status_code}')
          continue  # Stop searching further pages on HTTP error
      
      print(f'Month {month} over.')
    print(f'Year {year} over.')

  for jobs in jobs_from_all_dates:
    merged_jobs.extend(jobs)

  count_of_merged_jobs = len(merged_jobs)

  print(f'Count of merged jobs: {count_of_merged_jobs}')  
  return merged_jobs

all_jobs = aggregate_job_postings()
df = pd.DataFrame(all_jobs)
df.to_csv('job_postings.csv', index=False)

Found https://opportunitiesforyoungkenyans.co.ke/2018/1/1/page/1
Error 404 found on https://opportunitiesforyoungkenyans.co.ke/2018/1/1/page/2. Stopping search.
Day 1 over.
Found https://opportunitiesforyoungkenyans.co.ke/2018/1/2/page/1
Error 404 found on https://opportunitiesforyoungkenyans.co.ke/2018/1/2/page/2. Stopping search.
Day 2 over.
Found https://opportunitiesforyoungkenyans.co.ke/2018/1/3/page/1
Error 404 found on https://opportunitiesforyoungkenyans.co.ke/2018/1/3/page/2. Stopping search.
Day 3 over.
Found https://opportunitiesforyoungkenyans.co.ke/2018/1/4/page/1
Error 404 found on https://opportunitiesforyoungkenyans.co.ke/2018/1/4/page/2. Stopping search.
Day 4 over.
Found https://opportunitiesforyoungkenyans.co.ke/2018/1/5/page/1
Found https://opportunitiesforyoungkenyans.co.ke/2018/1/5/page/2
Error 404 found on https://opportunitiesforyoungkenyans.co.ke/2018/1/5/page/3. Stopping search.
Day 5 over.
Found https://opportunitiesforyoungkenyans.co.ke/2018/1/6/page/1
Error

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))